# EDA_75
# Guía Práctica para analizar las elecciones en USA (1976-2010) con Pandas

Usando Pandas para extraer **insights** (percepciones) de las elecciones.

Original: https://towardsdatascience.com/analysis-of-us-elections-from-1976-to-2010-with-pandas-6654cf69849d

Dataset: https://www.kaggle.com/tunguz/us-elections-dataset (EDA_75_president-1976-2016.csv)

USA prepara sus elecciones y en kaggle disponemos de un dataset para hacer análisis de las anteriores elecciones presidenciales de Estados Unidos.

El conjunto de datos contiene elecciones de 1976 a 2016. Abordaremos los datos desde diferentes perspectivas y trataremos de  obtener una idea de cómo votó la gente.

Usaremos **Pandas** para el análisis y visualización de datos, por lo que también será una buena práctica usar las funciones y métodos de Pandas.

Comencemos por importar las bibliotecas y leer el conjunto de datos en un **dataframe** de Pandas.

In [ ]:
import numpy as np
import pandas as pd

A continuación, leamos los datos del dataset.

In [ ]:
president = pd.read_csv("EDA_35_president-1976-2016.csv")

Veamos las primeras filas usando la función `head()`

In [ ]:
print('Primeras filas del dataset: \n', president.head())
print('\n')
print('Últimas filas del dataset: \n', president.tail())

Hay algunas columnas redundantes para el análisis. No estoy seguro de qué significan `state_fips`, `state_cen` y `state_ic`, pero pueden servir como un indicador o un valor único para un estado.

Podemos confirmar comprobando y comparando los valores de estas columnas. Estas tres columnas tienen 51 valores únicos (uno para cada estado).

In [ ]:
president.state_fips.nunique()

Para un estado en particular, los valores en estas columnas son los mismos:

In [ ]:
president[president.state == 'Alabama'][['state_fips','state_cen','state_ic']].value_counts()

Dado que el conjunto de datos ya contiene información de estado, estas columnas son redundantes en términos de análisis, por lo que pueden descartarse.

La columna de la oficina solo indica que esta es la elección presidencial, por lo que contiene un valor único (presidente de EE. UU.). Las columnas de versión y notas tampoco sirven de nada.

Podemos soltar estas columnas usando la función `drop` de Pandas.

In [ ]:
president.drop(['state_po', 'state_fips', 'state_cen', 'state_ic','office', 'version', 'notes'], axis=1, inplace=True)
president.head()

Estamos listos para comenzar el análisis que dividí en tres secciones:

- Número total de votos a lo largo del tiempo
- Las proporciones de los votos para cada ganador.
- Comparación estatal de partidos

## A: Número Total de Votos

Lo primero que comprobaremos es el número de votos por elección. La columna `totalvotes` muestra el número total de votos en un estado en particular. Por lo tanto, el siguiente código creará un marco de datos que contiene el total de votos en cada estado para cada elección.

In [ ]:
yearly_votes = president[['year','state','totalvotes']].groupby(['year','state']).mean().reset_index()
yearly_votes.head()

Podemos aplicar la función groupby en la columna `año` y sumar los valores en la columna `totalvotes`, lo que nos dará el número total de votos por elección.

In [ ]:
yearly_votes.groupby('year').sum().plot(figsize=(10,6), legend=False,title='Total Number of Votes in US Presendial Elections')

Usé las herramientas de trazado de Pandas para trazar los resultados. Es más simple que usar la interfaz pyplot de Matplotlib, pero tiene menos control sobre la trama.

El número de personas que votaron ha aumentado de manera constante, excepto en 1996 y 2012. Es muy posible que se deba al aumento de la población, por lo que una mejor medida sería el número de votos por millón de habitantes (elegibles para votar).

Como comparación aproximada, la población ha aumentado aproximadamente un 50% desde 1970 (203 millones) hasta 2010 (308 millones). Por lo tanto, la **proporción de personas que votan se ha mantenido aproximadamente igual a lo largo de los años.**

## B: Ratio de Votos por cada Ganador

Algunas elecciones fueron tan reñidas que el ganador ganó por un pequeño porcentaje. También hubo algunas elecciones en las que el ganador ganó por un gran margen.
Podemos calcular el ratio de votos de cada ganador. Primero agregaremos una columna `winner` a nuestro marco de datos (dataframe).

Esta página de Wikipedia (https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States) contiene una lista de presidentes de Estados Unidos. Las tablas se pueden leer fácilmente en un marco de datos de pandas utilizando la función `read_html`. Convierte las tablas de la página web en una lista de marcos de datos.

In [ ]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States")
winners = dfs[1][['Election','President.1']]
winners.head()

El segundo marco de datos contiene la lista de presidentes de Estados Unidos. Solo necesitamos los datos de las elecciones de 1976 a 2016.

In [ ]:
winners = winners.iloc[-12:-1, :]
winners.Election = winners.Election.astype('int64')
winners.rename(columns={'President.1':'winner'}, inplace=True)
winners

Necesitamos los nombres en el mismo formato que en el marco de datos del presidente. *Jimmy Carter* debe formatearse como *Carter, Jimmy*. Usaré las operaciones de cadena de pandas para hacer esta tarea:

In [ ]:
first_name = winners.winner.str.rsplit(' ', n=1, expand=True)[0]
last_name = winners.winner.str.rsplit(' ', n=1, expand=True)[1]
winners.winner = last_name + ', ' + first_name
winners

Necesitamos algunos pequeños ajustes para que los nombres de los presidentes coincidan exactamente.

In [ ]:
winners.winner[73] = 'Bush, George H.W.'
winners.winner[78] = 'Obama, Barack H.'
winners.winner[79] = 'Obama, Barack H.'
winners.winner[80] = 'Trump, Donald J.'

También convertí las fechas de las elecciones a números enteros para poder usar la función de combinación en el siguiente paso.
Ahora podemos fusionar el marco de datos de `president` y `winners` según el año electoral.

In [ ]:
president = pd.merge(president, winners, left_on='year', right_on='Election')

Filtraremos el marco de datos del presidente para incluir solo los votos de los ganadores.

In [ ]:
winner_votes = president[president.candidate == president.winner]
winner_votes.head()

Cada línea contiene el número de votos del ganador y el total de votos en un estado en particular para una elección en particular. Una simple función `groupby` nos dará los valores por país.

In [ ]:
total_votes = winner_votes[['year','winner','candidatevotes','totalvotes']].groupby(['year','winner']).sum()
total_votes

Podemos calcular la proporción de los ganadores mediante una simple operación matemática y ordenar los resultados.

In [ ]:
(total_votes.candidatevotes / total_votes.totalvotes).sort_values(ascending=False)

El número uno es Ronald Reagan en su segundo mandato.

## C: Comparación estatal de partes

Si te gusta la política, tienes una idea sobre a qué partido es probable que cada estado apoye más. También comprobemos eso con los números.

This is the winner_votes dataframe. We will add a ratio column which is candidate votes divided by total votes.

In [ ]:
winner_votes['ratio'] = winner_votes.candidatevotes / winner_votes.totalvotes

Queremos ver los diez mejores estados para demócratas y republicanos. La siguiente operación grupal devolverá los diez primeros estados según la proporción promedio más alta para los demócratas.

In [ ]:
winner_votes[winner_votes.party == 'democrat'][['state','ratio']]\
.groupby('state').mean().sort_values(by='ratio', ascending=False)[:10]

Aquí está el resultado de la misma operación para el bando republicano.

In [ ]:
winner_votes[winner_votes.party == 'republican'][['state','ratio']]\
.groupby('state').mean().sort_values(by='ratio', ascending=False)[:10]

## Conclusión

Hemos analizado las elecciones presidenciales de Estados Unidos con respecto al número de votos a lo largo del tiempo, el dominio de cada presidente en términos de votos y los votos estatales para demócratas y republicanos. Los resultados pueden no sorprenderte, especialmente si te gusta la política.
El otro enfoque de esta publicación es practicar cómo se pueden usar Pandas en el análisis y la manipulación de datos. Hemos realizado una gran cantidad de operaciones de Pandas en términos de análisis y manipulación de datos.